In [1]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb
%run /home/dbaciur/NTU/NTU/notes/CommonUtils.ipynb

Constants loaded


In [2]:
import torch
import torchvision as tv

import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
import intervals as I
import moviepy
from pathlib import Path
from moviepy.editor import VideoFileClip

### Load frames and events

In [3]:
frames_df = pd.read_csv(C.Frames.FRAMES_DF_DRAFT_PATH, converters=C.F_CONVERTERS)
frames_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layouts,Layout
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]",3.0,3
1,PH1011-PHYSICS_20150922__3703.298__43__B_CHART...,PH1011-PHYSICS_20150922,3703.298,43,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3
2,PH1011-PHYSICS_20150922__4040.803__44__B_CHART...,PH1011-PHYSICS_20150922,4040.803,44,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3
3,PH1011-PHYSICS_20150922__4284.434__48__B_CHART...,PH1011-PHYSICS_20150922,4284.434,48,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3
4,PH1011-PHYSICS_20150922__4957.804__62__B_CHART...,PH1011-PHYSICS_20150922,4957.804,62,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]",3.0,3


In [4]:
events_df = pd.read_csv(C.PROCESSED_ANNOTATIONS_PATH, converters=C.A_CONVERTERS)
events_df.head()

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video,Time,Start H (s),Stop H (s),Time H
0,0,0,eye contact,STATE,Encouraging learner engagement,0.0,0,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_EYE,PH1011-PHYSICS_20150922,1449.859,00:00:29:355,00:47:50:363,00:24:09:859
1,1,1,Writing on slides,STATE,Providing clear explanation; Arousing interest,0.0,0,2739.504,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_WRITING_SLIDES,PH1011-PHYSICS_20150922,1502.103,00:02:12:351,00:47:51:855,00:25:02:103
2,2,2,use of voice intonation to Emphasise more impo...,STATE,Encouraging learner engagement; Pacing and mai...,0.0,0,4.002,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_VOICE,PH1011-PHYSICS_20150922,352.397,00:05:50:396,00:05:54:398,00:05:52:397
3,3,3,"S, Asking questions",STATE,Activating prior knowledge; Encouraging learne...,0.0,0,29.062,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_ASK_QUESTIONS_S,PH1011-PHYSICS_20150922,372.325,00:05:57:794,00:06:26:856,00:06:12:325
4,4,4,"P, Asking questions",POINT,Activating prior knowledge; Encouraging learne...,0.0,0,0.000,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_ASK_QUESTIONS_P,PH1011-PHYSICS_20150922,358.794,00:05:58:794,00:05:58:794,00:05:58:794


### Add layout information

In [9]:
frame_events_ids = frames_df[C.F_EVENT_ID].unique()
len(frame_events_ids)

4920

In [10]:
frame_events_df = events_df[events_df[C.A_EVENT_ID].isin(frame_events_ids)]
len(frame_events_df)

4920

In [19]:
videos_per_subject = {}

for _, row in tqdm(frame_events_df.iterrows()):
    video = row[C.A_VIDEO]
    subject = row[C.A_SUBJECT_ID]
    
    if subject not in videos_per_subject:
        videos_per_subject[subject] = set()
    videos_per_subject[subject].add(video)

for key, value in videos_per_subject.items():
    videos_per_subject[key] = list(value)

In [20]:
import json as j
print(j.dumps(videos_per_subject, indent=2))

{
  "J1S1C11": [
    "PH1011-PHYSICS_20151020",
    "PH1011-PHYSICS_20150922",
    "PH1011-PHYSICS_20151013"
  ],
  "J1S1C13": [
    "PH1104-MECHANICS_20150817_PC1"
  ],
  "J1S1C14": [
    "15S1-PH1105-LEC_20150828",
    "15S1-PH1105-LEC_20150824",
    "15S1-PH1105-LEC_20150821",
    "15S1-PH1105-LEC_20150817"
  ],
  "J1S1C5": [
    "15S1-MH1800-LEC_20150831",
    "15S1-MH1800-LEC_20150817",
    "15S1-MH1800-LEC_20150824"
  ],
  "J1S2C15": [
    "15S2-PH1106-LEC_20160127",
    "15S2-PH1106-LEC_20160210",
    "15S2-PH1106-LEC_20160113",
    "15S2-PH1106-LEC_20160120",
    "15S2-PH1106-LEC_20160203"
  ],
  "J1S2C4": [
    "15S2-MH1402-LEC_20160115",
    "15S2-MH1402-LEC_20160205",
    "15S2-MH1402-LEC_20160122",
    "15S2-MH1402-LEC_20160129"
  ],
  "J2S1C10": [
    "16S1-MH1812-LEC_20160822",
    "16S1-MH1812-LEC_20160808",
    "16S1-MH1812-LEC_20160818",
    "16S1-MH1812-LEC_20160811"
  ],
  "J2S1C13": [
    "16S1-PH1104-LEC_20161107",
    "16S1-PH1104-LEC_20161031",
    "16S1-PH1104-L

In [22]:
subjects = {
  "J1S1C11": [
    "PH1011-PHYSICS_20151020",
    "PH1011-PHYSICS_20150922",
    "PH1011-PHYSICS_20151013"
  ],
  "J1S1C13": [
    "PH1104-MECHANICS_20150817_PC1"
  ],
  "J1S1C14": [
    "15S1-PH1105-LEC_20150828",
    "15S1-PH1105-LEC_20150824",
    "15S1-PH1105-LEC_20150821",
    "15S1-PH1105-LEC_20150817"
  ],
  "J1S1C5": [
    "15S1-MH1800-LEC_20150831",
    "15S1-MH1800-LEC_20150817",
    "15S1-MH1800-LEC_20150824"
  ],
  "J1S2C15": [
    "15S2-PH1106-LEC_20160127",
    "15S2-PH1106-LEC_20160210",
    "15S2-PH1106-LEC_20160113",
    "15S2-PH1106-LEC_20160120",
    "15S2-PH1106-LEC_20160203"
  ],
  "J1S2C4": [
    "15S2-MH1402-LEC_20160115",
    "15S2-MH1402-LEC_20160205",
    "15S2-MH1402-LEC_20160122",
    "15S2-MH1402-LEC_20160129"
  ],
  "J2S1C10": [
    "16S1-MH1812-LEC_20160822",
    "16S1-MH1812-LEC_20160808",
    "16S1-MH1812-LEC_20160818",
    "16S1-MH1812-LEC_20160811"
  ],
  "J2S1C13": [
    "16S1-PH1104-LEC_20161107",
    "16S1-PH1104-LEC_20161031",
    "16S1-PH1104-LEC_20161103",
    "16S1-PH1104-LEC_20161110",
    "16S1-PH1104-LEC_20161024"
  ],
  "J2S1C17": [
    "16S1-PH1801-LEC_20161020",
    "16S1-PH1801-LEC_20161103",
    "16S1-PH1801-LEC_20161013",
    "16S1-PH1801-LEC_20161110",
    "16S1-PH1801-LEC_20161027"
  ],
  "J2S2C3": [
    "16S2-MH1301-LEC_20170206",
    "16S2-MH1301-LEC_20170213",
    "16S2-MH1301-LEC_20170123",
    "16S2-MH1301-LEC_20170109",
    "16S2-MH1301-LEC_20170116"
  ],
  "J3S1C11": [
    "17S1-PH1011-LEC_20171017",
    "17S1-PH1011-LEC_20170912",
    "17S1-PH1011-LEC_20171031",
    "17S1-PH1011-LEC_20171024",
    "17S1-PH1011-LEC_20171010"
  ],
  "J3S1C17": [
    "17S1-PH1801-LEC_20170831",
    "17S1-PH1801-LEC_20170914",
    "17S1-PH1801-LEC_20170907",
    "17S1-PH1801-LEC_20170824",
    "17S1-PH1801-LEC_20170921",
    "17S1-PH1801-LEC_20170817"
  ],
  "J3S1C1": [
    "17S1-CM1021-LEC_20170906",
    "17S1-CM1021-LEC_20170823"
  ],
  "J3S1C2": [
    "17S1-MH1100-LEC_20170831",
    "17S1-MH1100-LEC_20171020",
    "17S1-MH1100-LEC_20171027",
    "17S1-MH1100-LEC_20170824",
    "17S1-MH1100-LEC_20171026",
    "17S1-MH1100-LEC_20170817",
    "17S1-MH1100-LEC_20170818",
    "17S1-MH1100-LEC_20170825",
    "17S1-MH1100-LEC_20171012",
    "17S1-MH1100-LEC_20171019"
  ],
  "J3S2C16": [
    "17S2-PH1107-CY1307-LEC_20180220",
    "17S2-PH1107-CY1307-LEC_20180130",
    "17S2-PH1107-CY1307-LEC_20180116",
    "17S2-PH1107-CY1307-LEC_20180123",
    "17S2-PH1107-CY1307-LEC_20180206"
  ],
  "J3S2C18": [
    "17S2-PH1802-LEC_20180413",
    "17S2-PH1802-LEC_20180406",
    "17S2-PH1802-LEC_20180420",
    "17S2-PH1802-LEC_20180324"
  ],
  "J3S2C3": [
    "17S2-MH1301-LEC_20180205",
    "17S2-MH1301-LEC_20180122",
    "17S2-MH1301-LEC_20180129",
    "17S2-MH1301-LEC_20180115",
    "17S2-MH1301-LEC_20180219"
  ],
  "J3S2C7": [
    "17S2-MH1803-LEC_20180130",
    "17S2-MH1803-LEC_20180117",
    "17S2-MH1803-LEC_20180116",
    "17S2-MH1803-LEC_20180131",
    "17S2-MH1803-LEC_20180124",
    "17S2-MH1803-LEC_20180123"
  ],
  "J3S2C8": [
    "17S2-MH1804-LEC_20180419",
    "17S2-MH1804-LEC_20180412",
    "17S2-MH1804-LEC_20180315",
    "17S2-MH1804-LEC_20180329"
  ],
  "J3S2C9": [
    "17S2-MH1811-LEC_20180214",
    "17S2-MH1811-LEC_20180212",
    "17S2-MH1811-LEC_20180221",
    "17S2-MH1811-LEC_20180205",
    "17S2-MH1811-LEC_20180219"
  ],
  "J4S1C2": [
    "18S1-MH1100-LEC_20181026",
    "18S1-MH1100-LEC_20180914",
    "18S1-MH1100-LEC_20180913",
    "18S1-MH1100-LEC_20180831",
    "18S1-MH1100-LEC_20181011",
    "18S1-MH1100-LEC_20181025",
    "18S1-MH1100-LEC_20181019",
    "18S1-MH1100-LEC_20180907",
    "18S1-MH1100-LEC_20181012",
    "18S1-MH1100-LEC_20181018"
  ],
  "J4S1C6": [
    "18S1-MH1802-LEC_20180829",
    "18S1-MH1802-LEC_20180827",
    "18S1-MH1802-LEC_20180820"
  ]
}